In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import torch
import torch.nn as nn

import torch.nn.functional as F
import dgl
import dgl.nn as dglnn
import torch.nn as nn
from dgl import AddSelfLoop

from model_2d.NMR_gcn import NMR_GCN

from train_evaluate_2d import NMR_prediction

from preprocess.create_node_embedding.node_embeddings_glycoscience import create_node_embeddings

from preprocess.build_adjaency_matrix.create_adjaency_matrix_glycoscience import build_adjacency_matrix

from preprocess.create_graph.create_graph_data_glycosciencedb import create_graph

# from dgl import save_graphs, load_graphs
# from tqdm import tqdm
from dgl import save_graphs, load_graphs


##### 1, Create Node embeddings

##### Initialize atom-level node embeddings based on extracted atom and monosaccharide features. 

##### Due to the inconsistent of data formating, as GlycoscienceDB is retrieved from various labs, the method of creating node embedding is slighly different from GODESS.

1, atom_dim: dimension of the atom name that indicate ring position, Eg: 04, C4, C3...

2, mono_dim: dimension of the monosaccharide name, glc.

3, ab_dim: dimension of the anomeric orientation of hydroxyl group.

4, dl_dim: dimension of the fischer convention, fischer d, l.

5, pf_dim: dimension of the number of carbon p, f.

6, residual_dim: dimension of the residual number starting from the root as number 1.



In [2]:
atom_dim = 256

residual_dim = 128

mono_dim = 64
ab_dim = 64
dl_dim = 64
pf_dim = 64

seed=9721

In [3]:
C = create_node_embeddings(seed=seed)

df_atom_embedding, df_residual_embedding, df_monosaccharide_embedding, df_ab_embedding, df_dl_embedding, df_pf_embedding =\
C.create_all_embeddings(atom_dim=atom_dim, residual_dim=residual_dim, mono_dim=mono_dim, 
                       ab_dim=ab_dim, dl_dim=dl_dim, pf_dim=pf_dim)



100%|████████████████████████████████████████| 299/299 [00:00<00:00, 661.45it/s]


In [4]:
df_atom_embedding.to_csv(C.out_atom_embed, index=False)
df_residual_embedding.to_csv(C.out_residual_embed, index=False)
df_monosaccharide_embedding.to_csv(C.out_monosaccharide_embed, index=False)

df_ab_embedding.to_csv(C.out_bound_ab, index = False) 
df_dl_embedding.to_csv(C.out_bound_dl, index = False) 
df_pf_embedding.to_csv(C.out_carbon_pf, index = False) 

##### 2, Create adjaency matrix

##### Build adjaency matrix based on threshold distance between atoms

1, threshold_carbon: we construct edges between two carbon atom if their distance is within 1.65Å. 

2, threshold_hydrogen: we construct edges between two hydrogen atom if their distance is within 1.18Å. 

3, threshold_general: for all other atoms, we construct edges between distance two atom is within 1.5Å. 

4, threshold_interaction: we construct interaction edges between atoms if their distance is with in 5Å. 


In [5]:
B = build_adjacency_matrix()

In [6]:
B.calculate_all_matrix()

100%|█████████████████████████████████████████| 299/299 [00:04<00:00, 63.82it/s]


##### 3, Create glycan graph

##### create glycan graph from prebuilt node embeddings and adjaency matrix, every glycan is considered as a graph and every atom is consided as a node

1, num_test: number of testing glycans, we randomly select 20% of the glycans as test set


In [7]:
num_test = 60 # 299 * 20% 

In [8]:
Create = create_graph(num_test=num_test, seed=seed)
g, test_index = Create.create_all_graph()

--------------------------loading NMR Graph-------------------------------


100%|█████████████████████████████████████████| 299/299 [00:05<00:00, 54.44it/s]


##### 4, Train a 4-layer GNN model

In [9]:
num_epoch = 1000
lr = 1e-2


In [10]:
in_size = atom_dim + mono_dim + ab_dim + dl_dim + pf_dim


hidden_size_1 = int(in_size / 2)
hidden_size_2 = 256
hidden_size_3 = 128
hidden_size_4 = 32

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

g = g.to(device)
features = g.ndata["feat"]
labels = g.ndata["shift_value"]
# masks = g.ndata['train_mask'], g.ndata['test_mask']


# masks = g.ndata['train_hydrogen_mask'], g.ndata['test_hydrogen_mask']
masks = g.ndata['train_carbon_mask'], g.ndata['test_carbon_mask']


print(features.dtype)
print(labels.dtype)
# model = NMR_GCN(in_size=576, hid_size=[256, 128, 64, 32], out_size=1).to(device)
model = NMR_GCN(in_size=in_size, hid_size=[hidden_size_1, hidden_size_2,
                                       hidden_size_3, hidden_size_4], out_size=1).to(device)
# model training

# NMR_prediction = NMR_prediction(results_dir='experimental_data_combined/graph_combined/experimental_results_all_hydrogen.csv',
#                                 model_dir='experimental_data_combined/graph_combined/Model_hydrogen_experiment_training.pt',
#                                num_epoch = num_epoch, 
#                                lr = lr)


NMR_prediction = NMR_prediction(results_dir='glycosciencedb/results/training_carbon.csv',
                                results_dir_test = 'glycosciencedb/results/testing_carbon.csv',
                                model_dir='glycosciencedb/results/Model_Godess_carbon.pt', 
                               num_epoch = num_epoch, 
                               lr = lr)


print("Training...")
NMR_prediction.train(g, features, labels, masks, model)

# test the model
print("Testing...")
saved_model = NMR_GCN(in_size=in_size, hid_size=[hidden_size_1, hidden_size_2,
                                                 hidden_size_3, hidden_size_4], out_size=1).to(device)
saved_model.load_state_dict(torch.load(NMR_prediction.model_dir))

# acc = NMR_prediction.evaluate(g, features, labels, masks[0], saved_model, print_out=True)
acc1 = NMR_prediction.evaluate(g, features, labels, masks[0], saved_model, save_train=True, save_test = False)

acc2 = NMR_prediction.evaluate(g, features, labels, masks[1], saved_model, save_train=False, save_test = True)


print("train RMSE {:.4f}".format(acc1))
print("test RMSE {:.4f}".format(acc2))

torch.float32
torch.float32
Training...


  1%|▍                                        | 10/1000 [00:00<01:02, 15.86it/s]

1247
5263
Epoch 00000 | Loss 6159.6245 | train_RMSE 62.9367 | test_RMSE 64.1123 
1247
5263
Epoch 00001 | Loss 3964.5342 | train_RMSE 121.3373 | test_RMSE 122.8760 
1247
5263
Epoch 00002 | Loss 14775.7119 | train_RMSE 41.9920 | test_RMSE 42.8760 
1247
5263
Epoch 00003 | Loss 1765.9730 | train_RMSE 66.4841 | test_RMSE 67.7287 
1247
5263
Epoch 00004 | Loss 4419.2017 | train_RMSE 68.9947 | test_RMSE 70.2361 
1247
5263
Epoch 00005 | Loss 4758.8799 | train_RMSE 62.1200 | test_RMSE 63.2098 
1247
5263
Epoch 00006 | Loss 3855.2688 | train_RMSE 38.5400 | test_RMSE 39.0804 
1247
5263
Epoch 00007 | Loss 1489.3617 | train_RMSE 48.6284 | test_RMSE 49.0380 
1247
5263
Epoch 00008 | Loss 2381.9917 | train_RMSE 38.4136 | test_RMSE 38.6851 
1247
5263
Epoch 00009 | Loss 1503.2722 | train_RMSE 23.2776 | test_RMSE 23.5405 
1247
5263
Epoch 00010 | Loss 553.0962 | train_RMSE 36.8670 | test_RMSE 37.5717 
1247
5263
Epoch 00011 | Loss 1367.2859 | train_RMSE 34.4086 | test_RMSE 35.0795 
1247
5263
Epoch 00012 | Lo

  3%|█▏                                       | 29/1000 [00:01<00:22, 43.84it/s]

1247
5263
Epoch 00018 | Loss 750.3179 | train_RMSE 22.3996 | test_RMSE 23.1538 
1247
5263
Epoch 00019 | Loss 513.5949 | train_RMSE 16.3721 | test_RMSE 16.9480 
1247
5263
Epoch 00020 | Loss 283.9879 | train_RMSE 22.3180 | test_RMSE 22.6694 
1247
5263
Epoch 00021 | Loss 533.5200 | train_RMSE 21.7701 | test_RMSE 22.1067 
1247
5263
Epoch 00022 | Loss 493.7584 | train_RMSE 15.4770 | test_RMSE 16.0651 
1247
5263
Epoch 00023 | Loss 255.3528 | train_RMSE 18.6189 | test_RMSE 19.4578 
1247
5263
Epoch 00024 | Loss 363.0672 | train_RMSE 20.7746 | test_RMSE 21.6794 
1247
5263
Epoch 00025 | Loss 446.3779 | train_RMSE 17.4507 | test_RMSE 18.3269 
1247
5263
Epoch 00026 | Loss 320.1314 | train_RMSE 14.2665 | test_RMSE 14.8876 
1247
5263
Epoch 00027 | Loss 225.1390 | train_RMSE 17.5952 | test_RMSE 17.8969 
1247
5263
Epoch 00028 | Loss 325.0063 | train_RMSE 17.1380 | test_RMSE 17.4575 
1247
5263
Epoch 00029 | Loss 316.2892 | train_RMSE 13.4042 | test_RMSE 14.0808 
1247
5263
Epoch 00030 | Loss 196.0741 | 

...

1247
5263
Epoch 00946 | Loss 21.9669 | train_RMSE 2.9568 | test_RMSE 3.2114 
1247
5263
Epoch 00947 | Loss 20.5179 | train_RMSE 2.9356 | test_RMSE 3.2433 
1247
5263
Epoch 00948 | Loss 20.9996 | train_RMSE 2.9960 | test_RMSE 3.3049 
1247
5263
Epoch 00949 | Loss 21.2365 | train_RMSE 2.9231 | test_RMSE 3.2113 
1247
5263
Epoch 00950 | Loss 21.1995 | train_RMSE 3.0743 | test_RMSE 3.3354 
1247
5263
Epoch 00951 | Loss 21.8220 | train_RMSE 3.0698 | test_RMSE 3.3476 
1247
5263
Epoch 00952 | Loss 21.9873 | train_RMSE 3.3234 | test_RMSE 3.5952 
1247
5263
Epoch 00953 | Loss 22.7693 | train_RMSE 3.2304 | test_RMSE 3.5077 
1247
5263
Epoch 00954 | Loss 23.6068 | train_RMSE 3.3518 | test_RMSE 3.6409 
1247
5263
Epoch 00955 | Loss 23.4312 | train_RMSE 3.3684 | test_RMSE 3.6098 
1247
5263
Epoch 00956 | Loss 24.5450 | train_RMSE 3.7585 | test_RMSE 4.0365 
1247
5263
Epoch 00957 | Loss 25.5337 | train_RMSE 3.6586 | test_RMSE 3.8921 
1247
5263
Epoch 00958 | Loss 25.9282 | train_RMSE 3.7665 | test_RMSE 4.0678 

 98%|███████████████████████████████████████▎| 983/1000 [00:11<00:00, 96.59it/s]

1247
5263
Epoch 00966 | Loss 24.3387 | train_RMSE 4.0168 | test_RMSE 4.2563 
1247
5263
Epoch 00967 | Loss 29.5872 | train_RMSE 4.9964 | test_RMSE 5.2180 
1247
5263
Epoch 00968 | Loss 36.5679 | train_RMSE 5.4319 | test_RMSE 5.6335 
1247
5263
Epoch 00969 | Loss 44.7918 | train_RMSE 7.2235 | test_RMSE 7.4825 
1247
5263
Epoch 00970 | Loss 63.2851 | train_RMSE 6.7547 | test_RMSE 6.9905 
1247
5263
Epoch 00971 | Loss 59.8983 | train_RMSE 7.4515 | test_RMSE 7.6437 
1247
5263
Epoch 00972 | Loss 65.5357 | train_RMSE 5.3967 | test_RMSE 5.6071 
1247
5263
Epoch 00973 | Loss 43.2592 | train_RMSE 4.2784 | test_RMSE 4.5761 
1247
5263
Epoch 00974 | Loss 29.5175 | train_RMSE 3.1419 | test_RMSE 3.4609 
1247
5263
Epoch 00975 | Loss 23.6318 | train_RMSE 3.1747 | test_RMSE 3.4495 
1247
5263
Epoch 00976 | Loss 21.9631 | train_RMSE 4.1961 | test_RMSE 4.3729 
1247
5263
Epoch 00977 | Loss 28.8229 | train_RMSE 4.1381 | test_RMSE 4.3054 
1247
5263
Epoch 00978 | Loss 30.9576 | train_RMSE 4.2422 | test_RMSE 4.5385 

100%|███████████████████████████████████████| 1000/1000 [00:11<00:00, 88.31it/s]


1247
5263
Epoch 00986 | Loss 30.5411 | train_RMSE 3.6400 | test_RMSE 3.8406 
1247
5263
Epoch 00987 | Loss 27.4270 | train_RMSE 3.5325 | test_RMSE 3.7413 
1247
5263
Epoch 00988 | Loss 24.5160 | train_RMSE 3.0581 | test_RMSE 3.2506 
1247
5263
Epoch 00989 | Loss 22.6591 | train_RMSE 2.9938 | test_RMSE 3.2310 
1247
5263
Epoch 00990 | Loss 21.5299 | train_RMSE 3.6314 | test_RMSE 3.9538 
1247
5263
Epoch 00991 | Loss 25.3893 | train_RMSE 3.8088 | test_RMSE 3.9799 
1247
5263
Epoch 00992 | Loss 27.7549 | train_RMSE 4.1715 | test_RMSE 4.4737 
1247
5263
Epoch 00993 | Loss 29.9047 | train_RMSE 3.9140 | test_RMSE 4.0863 
1247
5263
Epoch 00994 | Loss 28.7314 | train_RMSE 3.7736 | test_RMSE 4.0736 
1247
5263
Epoch 00995 | Loss 26.0431 | train_RMSE 3.1477 | test_RMSE 3.4274 
1247
5263
Epoch 00996 | Loss 22.7853 | train_RMSE 2.9474 | test_RMSE 3.2709 
1247
5263
Epoch 00997 | Loss 21.7370 | train_RMSE 3.2629 | test_RMSE 3.5241 
1247
5263
Epoch 00998 | Loss 23.9562 | train_RMSE 4.1695 | test_RMSE 4.3962 